In [8]:
import pandas as pd
from sqlalchemy import create_engine

# Reemplaza estos valores con los detalles de tu base de datos
username = 'root'
password = 'Custom1234'
host = 'localhost'
port = '3306'
database = 'sakila'

# Crear la conexión a la base de datos
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')


### 1. Calcular la duración media del alquiler (en días) para cada película:

In [9]:
query = """
SELECT 
    f.title, 
    f.rental_duration, 
    AVG(DATEDIFF(r.return_date, r.rental_date)) AS avg_rental_duration
FROM 
    film f
JOIN 
    inventory i ON f.film_id = i.film_id
JOIN 
    rental r ON i.inventory_id = r.inventory_id
GROUP BY 
    f.title, f.rental_duration;
"""
result = pd.read_sql_query(query, engine)
result.head()

,title,rental_duration,avg_rental_duration
0,ACADEMY DINOSAUR,6,5.0909
1,ACE GOLDFINGER,3,5.6667
2,ADAPTATION HOLES,7,3.4167
3,AFFAIR PREJUDICE,5,4.7273
4,AFRICAN EGG,6,7.0909


|title|rental_duration|avg_rental_duration|
|-----|---------------|-------------------|
|ACADEMY DINOSAUR|6              |4.9850             |
|ACE GOLDFINGER|3              |4.9850             |
|ADAPTATION HOLES|7              |4.9850             |
|AFFAIR PREJUDICE|5              |4.9850             |
|AFRICAN EGG|6              |4.9850             |
|AGENT TRUMAN|3              |4.9850             |
|AIRPLANE SIERRA|6              |4.9850             |
|AIRPORT POLLOCK|6              |4.9850             |
|ALABAMA DEVIL|3              |4.9850             |
|ALADDIN CALENDAR|6              |4.9850             |
|ALAMO VIDEOTAPE|6              |4.9850             |
|ALASKA PHANTOM|6              |4.9850             |
|ALI FOREVER|4              |4.9850             |
|ALICE FANTASIA|6              |4.9850             |


### 2. Calcular el importe medio de los pagos para cada miembro del personal:

In [13]:
query = """
SELECT 
    staff_id, 
    AVG(amount) AS avg_payment_amount
FROM 
    payment
GROUP BY 
    staff_id;
"""
result = pd.read_sql_query(query, engine)
result.head()

,staff_id,avg_payment_amount
0,1,4.157251
1,2,4.245815


#### Output

|staff_id|avg_payment_amount|
|--------|------------------|
|1       |4.156568          |
|1       |4.156568          |
|1       |4.156568          |
|1       |4.156568          |
|1       |4.156568          |
|1       |4.156568          |
|1       |4.156568          |

### 3. Calcular los ingresos totales para cada cliente, mostrando el total acumulado dentro del historial de alquileres de cada cliente:

In [14]:
query = """
SELECT 
    p.customer_id,
    r.rental_id,
    r.rental_date,
    p.amount,
    SUM(p.amount) OVER(PARTITION BY p.customer_id ORDER BY r.rental_date) AS running_total
FROM 
    payment p
JOIN 
    rental r ON p.rental_id = r.rental_id;
"""
result = pd.read_sql_query(query, engine)
result.head(10)


,customer_id,rental_id,rental_date,amount,running_total
0,1,76,2005-05-25 11:30:37,2.99,2.99
1,1,573,2005-05-28 10:35:23,0.99,3.98
2,1,1185,2005-06-15 00:54:12,5.99,9.97
3,1,1422,2005-06-15 18:02:53,0.99,10.96
4,1,1476,2005-06-15 21:08:46,9.99,20.95
5,1,1725,2005-06-16 15:18:57,4.99,25.94
6,1,2308,2005-06-18 08:41:48,4.99,30.93
7,1,2363,2005-06-18 13:33:59,0.99,31.92
8,1,3284,2005-06-21 06:24:45,3.99,35.91
9,1,4526,2005-07-08 03:17:05,5.99,41.90


#### Output

|customer_id|rental_id|rental_date        |amount|running_total|
|-----------|---------|-------------------|------|-------------|
|1          |76       |2005-05-25 11:30:37|2.99  |2.99         |
|1          |573      |2005-05-28 10:35:23|0.99  |3.98         |
|1          |1185     |2005-06-15 00:54:12|5.99  |9.97         |
|1          |1422     |2005-06-15 18:02:53|0.99  |10.96        |
|1          |1476     |2005-06-15 21:08:46|9.99  |20.95        |
|1          |1725     |2005-06-16 15:18:57|4.99  |25.94        |
|1          |2308     |2005-06-18 08:41:48|4.99  |30.93        |
|1          |2363     |2005-06-18 13:33:59|0.99  |31.92        |
|1          |3284     |2005-06-21 06:24:45|3.99  |35.91        |
|1          |4526     |2005-07-08 03:17:05|5.99  |41.90        |

### 4. Determinar el cuartil para las tarifas de alquiler de las películas

In [15]:
query = """
WITH rental_rate_quartiles AS (
    SELECT 
        rental_rate,
        NTILE(4) OVER(ORDER BY rental_rate) AS quartile
    FROM 
        film
)
SELECT 
    f.title,
    f.rental_rate,
    rrq.quartile
FROM 
    film f
JOIN 
    rental_rate_quartiles rrq ON f.rental_rate = rrq.rental_rate;
"""
result = pd.read_sql_query(query, engine)
result.head(15)


,title,rental_rate,quartile
0,ACADEMY DINOSAUR,0.99,1
1,ACADEMY DINOSAUR,0.99,1
2,ACADEMY DINOSAUR,0.99,1
3,ACADEMY DINOSAUR,0.99,1
4,ACADEMY DINOSAUR,0.99,1
5,ACADEMY DINOSAUR,0.99,1
6,ACADEMY DINOSAUR,0.99,1
7,ACADEMY DINOSAUR,0.99,1
8,ACADEMY DINOSAUR,0.99,1
9,ACADEMY DINOSAUR,0.99,1


#### Output

|title|rental_rate|quartile           |
|-----|-----------|-------------------|
|ACADEMY DINOSAUR|0.99       |1                  |
|ALAMO VIDEOTAPE|0.99       |1                  |
|ALASKA PHANTOM|0.99       |1                  |
|ALICE FANTASIA|0.99       |1                  |
|ALONE TRIP|0.99       |1                  |
|ALTER VICTORY|0.99       |1                  |
|AMADEUS HOLY|0.99       |1                  |
|ANACONDA CONFESSIONS|0.99       |1                  |
|ANNIE IDENTITY|0.99       |1                  |
|ANONYMOUS HUMAN|0.99       |1                  |


### 5. Determinar la primera y última fecha de alquiler para cada cliente:

In [16]:
query = """
SELECT 
    customer_id,
    MIN(rental_date) AS first_rental_date,
    MAX(rental_date) AS last_rental_date
FROM 
    rental
GROUP BY 
    customer_id;
"""
result = pd.read_sql_query(query, engine)
result.head(10)

,customer_id,first_rental_date,last_rental_date
0,1,2005-05-25 11:30:37,2005-08-22 20:03:46
1,2,2005-05-27 00:09:24,2005-08-23 17:39:35
2,3,2005-05-27 17:17:09,2005-08-23 07:10:14
3,4,2005-06-15 09:31:28,2005-08-23 07:43:00
4,5,2005-05-29 07:25:16,2006-02-14 15:16:03
5,6,2005-05-25 08:43:32,2005-08-23 06:41:32
6,7,2005-05-25 06:04:08,2005-08-21 04:49:48
7,8,2005-05-30 03:43:54,2005-08-23 14:31:19
8,9,2005-05-27 05:01:28,2006-02-14 15:16:03
9,10,2005-05-31 19:36:30,2005-08-22 21:59:29


#### Expected result

|customer_id|first_rental_date|last_rental_date   |
|-----------|-----------------|-------------------|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|
|1          |2005-05-25 11:30:37|2005-08-22 20:03:46|


### 6. Calcular el rango de los clientes basado en el número de sus alquileres:

In [17]:
query = """
SELECT 
    customer_id,
    COUNT(rental_id) AS rental_count,
    DENSE_RANK() OVER(ORDER BY COUNT(rental_id) DESC) AS rental_count_rank
FROM 
    rental
GROUP BY 
    customer_id;
"""
result = pd.read_sql_query(query, engine)
result.head(10)


,customer_id,rental_count,rental_count_rank
0,148,46,1
1,526,45,2
2,144,42,3
3,236,42,3
4,75,41,4
5,197,40,5
6,469,40,5
7,137,39,6
8,178,39,6
9,468,39,6


#### Expected output

|customer_id|rental_count                 |rental_count_rank|
|-----------|-----------------------------|-----------------|
|148        |46                           |1                |
|526        |45                           |2                |
|144        |42                           |3                |
|236        |42                           |3                |
|75         |41                           |5                |
|197        |40                           |6                |
...|...|...

### 7. Calcular el total acumulado de ingresos por día para la categoría de películas 'Familiar':

In [18]:
query = """
SELECT 
    c.name AS film_category,
    r.rental_date,
    p.amount,
    SUM(p.amount) OVER(PARTITION BY c.name ORDER BY r.rental_date) AS daily_revenue
FROM 
    payment p
JOIN 
    rental r ON p.rental_id = r.rental_id
JOIN 
    inventory i ON r.inventory_id = i.inventory_id
JOIN 
    film f ON i.film_id = f.film_id
JOIN 
    film_category fc ON f.film_id = fc.film_id
JOIN 
    category c ON fc.category_id = c.category_id
WHERE 
    c.name = 'Family';
"""
result = pd.read_sql_query(query, engine)
result.head(10)


,film_category,rental_date,amount,daily_revenue
0,Family,2005-05-24 22:53:30,2.99,2.99
1,Family,2005-05-25 00:22:55,6.99,9.98
2,Family,2005-05-25 01:59:46,4.99,14.97
3,Family,2005-05-25 04:47:44,4.99,19.96
4,Family,2005-05-25 13:52:43,0.99,20.95
5,Family,2005-05-25 17:22:10,3.99,24.94
6,Family,2005-05-25 18:28:09,6.99,31.93
7,Family,2005-05-25 18:30:05,2.99,34.92
8,Family,2005-05-26 04:46:23,4.99,39.91
9,Family,2005-05-26 05:32:52,4.99,44.90


#### Expected ouput

|film_category|rental_date                  |amount|daily_revenue|
|-------------|-----------------------------|------|-------------|
|BLANKET BEVERLY|2005-05-24 22:53:30          |2.99  |2.99         |
|GRADUATE LORD|2005-05-24 23:03:39          |3.99  |3.99         |
|LAWLESS VISION|2005-05-24 23:31:46          |4.99  |4.99         |
|HANGING DEEP |2005-05-25 00:02:21          |5.99  |5.99         |
|MONTEREY LABYRINTH|2005-05-25 00:31:15          |0.99  |0.99         |
|SCISSORHANDS SLUMS|2005-05-25 01:48:41          |2.99  |2.99         |
|EASY GLADIATOR|2005-05-25 03:41:50          |4.99  |4.99         |

### 8. Asignar un ID único a cada pago dentro del historial de pagos de cada cliente:

In [19]:
query = """
SELECT 
    customer_id,
    payment_id,
    ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY payment_id) AS payment_sequence_id
FROM 
    payment;
"""
result = pd.read_sql_query(query, engine)
result.head(20)


,customer_id,payment_id,payment_sequence_id
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,1,5,5
5,1,6,6
6,1,7,7
7,1,8,8
8,1,9,9
9,1,10,10


|customer_id|payment_id|payment_sequence_id|
|-----------|----------|-------------------|
|1          |1         |1                  |
|1          |2         |2                  |
|1          |3         |3                  |
|1          |4         |4                  |
|1          |5         |5                  |
|1          |6         |6                  |
|1          |7         |7                  |
|1          |8         |8                  |
|1          |9         |9                  |
|1          |10        |10                 |
|1          |11        |11                 |
|1          |12        |12                 |
|1          |13        |13                 |
|1          |14        |14                 |
|1          |15        |15                 |
|1          |16        |16                 |
|1          |17        |17                 |
|1          |18        |18                 |
|1          |19        |19                 |
|1          |20        |20                 |
|1          |21        |21                 |
|1          |22        |22                 |
|1          |23        |23                 |
|1          |24        |24                 |
|1          |25        |25                 |
|1          |26        |26                 |
|1          |27        |27                 |
|1          |28        |28                 |
|1          |29        |29                 |
|1          |30        |30                 |
|1          |31        |31                 |
|1          |32        |32                 |
|2          |33        |1                  |
|2          |34        |2                  |
|2          |35        |3                  |
|2          |36        |4                  |
|2          |37        |5                  |
|2          |38        |6                  |
|2          |39        |7                  |
|2          |40        |8                  |
|2          |41        |9                  |
|2          |42        |10                 |
|2          |43        |11                 |
|2          |44        |12                 |


### 9. Calcular la diferencia en días entre cada alquiler y el alquiler anterior para cada cliente:

In [20]:
query = """
SELECT 
    customer_id,
    rental_id,
    rental_date,
    LAG(rental_date) OVER(PARTITION BY customer_id ORDER BY rental_date) AS previous_rental_date,
    DATEDIFF(rental_date, LAG(rental_date) OVER(PARTITION BY customer_id ORDER BY rental_date)) AS days_between_rentals
FROM 
    rental;
"""
result = pd.read_sql_query(query, engine)
result.head(10)

,customer_id,rental_id,rental_date,previous_rental_date,days_between_rentals
0,1,76,2005-05-25 11:30:37,NaT,NaN
1,1,573,2005-05-28 10:35:23,2005-05-25 11:30:37,3.0
2,1,1185,2005-06-15 00:54:12,2005-05-28 10:35:23,18.0
3,1,1422,2005-06-15 18:02:53,2005-06-15 00:54:12,0.0
4,1,1476,2005-06-15 21:08:46,2005-06-15 18:02:53,0.0
5,1,1725,2005-06-16 15:18:57,2005-06-15 21:08:46,1.0
6,1,2308,2005-06-18 08:41:48,2005-06-16 15:18:57,2.0
7,1,2363,2005-06-18 13:33:59,2005-06-18 08:41:48,0.0
8,1,3284,2005-06-21 06:24:45,2005-06-18 13:33:59,3.0
9,1,4526,2005-07-08 03:17:05,2005-06-21 06:24:45,17.0


#### Expected output


|customer_id|rental_id|rental_date        |previous_rental_date|days_between_rentals|
|-----------|---------|-------------------|--------------------|--------------------|
|1          |76       |2005-05-25 11:30:37|NULL                |NULL                |
|1          |573      |2005-05-28 10:35:23|2005-05-25 11:30:37 |3                   |
|1          |1185     |2005-06-15 00:54:12|2005-05-28 10:35:23 |18                  |
|1          |1422     |2005-06-15 18:02:53|2005-06-15 00:54:12 |0                   |
|1          |1476     |2005-06-15 21:08:46|2005-06-15 18:02:53 |0                   |
|1          |1725     |2005-06-16 15:18:57|2005-06-15 21:08:46 |1                   |
|1          |2308     |2005-06-18 08:41:48|2005-06-16 15:18:57 |2                   |
|1          |2363     |2005-06-18 13:33:59|2005-06-18 08:41:48 |0                   |
|1          |3284     |2005-06-21 06:24:45|2005-06-18 13:33:59 |3                   |
|1          |4526     |2005-07-08 03:17:05|2005-06-21 06:24:45 |17                  |
|1          |4611     |2005-07-08 07:33:56|2005-07-08 03:17:05 |0                   |

